## Išnagrinėkite K-Means klasterizavimą naudodami R ir Tidy duomenų principus.

### [**Klausimynas prieš paskaitą**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/29/)

Šioje pamokoje sužinosite, kaip kurti klasterius naudojant Tidymodels paketą ir kitus R ekosistemos paketus (juos vadinsime draugais 🧑‍🤝‍🧑), taip pat Nigerijos muzikos duomenų rinkinį, kurį importavote anksčiau. Aptarsime K-Means klasterizavimo pagrindus. Atminkite, kad, kaip sužinojote ankstesnėje pamokoje, yra daug būdų dirbti su klasteriais, o pasirinktas metodas priklauso nuo jūsų duomenų. Išbandysime K-Means, nes tai yra dažniausiai naudojama klasterizavimo technika. Pradėkime!

Sąvokos, apie kurias sužinosite:

-   Silueto įvertinimas

-   Alkūnės metodas

-   Inercija

-   Dispersija

### **Įvadas**

[K-Means klasterizavimas](https://wikipedia.org/wiki/K-means_clustering) yra metodas, kilęs iš signalų apdorojimo srities. Jis naudojamas duomenų grupėms padalyti ir suskirstyti į `k klasterius`, remiantis jų savybių panašumais.

Klasteriai gali būti vizualizuojami kaip [Voronoi diagramos](https://wikipedia.org/wiki/Voronoi_diagram), kurios apima tašką (arba „sėklą“) ir atitinkamą regioną.

<p >
   <img src="../../images/voronoi.png"
   width="500"/>
   <figcaption>Infografika, sukurta Jen Looper</figcaption>

K-Means klasterizavimas vyksta šiais etapais:

1.  Duomenų analitikas pradeda nurodydamas norimą klasterių skaičių.

2.  Tada algoritmas atsitiktinai pasirenka K stebėjimus iš duomenų rinkinio, kurie bus naudojami kaip pradiniai klasterių centrai (t. y. centroidai).

3.  Kiekvienas likęs stebėjimas priskiriamas artimiausiam centroidui.

4.  Tada apskaičiuojami nauji kiekvieno klasterio vidurkiai, o centroidas perkeliamas į vidurkį.

5.  Kai centrai perskaičiuoti, kiekvienas stebėjimas vėl patikrinamas, ar jis nėra arčiau kito klasterio. Visi objektai vėl priskiriami, naudojant atnaujintus klasterių vidurkius. Klasterių priskyrimo ir centroidų atnaujinimo žingsniai kartojami iteratyviai, kol klasterių priskyrimai nustoja keistis (t. y. pasiekiama konvergencija). Paprastai algoritmas baigiasi, kai kiekviena nauja iteracija sukelia nereikšmingą centroidų judėjimą, o klasteriai tampa statiški.

<div>

> Atkreipkite dėmesį, kad dėl pradinio k stebėjimų atsitiktinumo, naudojamų kaip pradiniai centroidai, kiekvieną kartą taikant procedūrą galime gauti šiek tiek skirtingus rezultatus. Dėl šios priežasties dauguma algoritmų naudoja kelis *atsitiktinius pradėjimus* ir pasirenka iteraciją su mažiausiu WCSS. Todėl primygtinai rekomenduojama visada vykdyti K-Means su keliais *nstart* reikšmėmis, kad būtų išvengta *nepageidaujamo lokalaus optimumo.*

</div>

Ši trumpa animacija, naudojant Allison Horst [iliustracijas](https://github.com/allisonhorst/stats-illustrations), paaiškina klasterizavimo procesą:

<p >
   <img src="../../images/kmeans.gif"
   width="550"/>
   <figcaption>Iliustracija, sukurta @allison_horst</figcaption>

Pagrindinis klausimas, kylantis klasterizavimo metu, yra toks: kaip žinoti, į kiek klasterių reikia suskirstyti duomenis? Vienas iš K-Means trūkumų yra tas, kad reikia nustatyti `k`, tai yra `centroidų` skaičių. Laimei, `alkūnės metodas` padeda įvertinti gerą pradinę `k` reikšmę. Netrukus tai išbandysite.

### 

**Būtinos sąlygos**

Pradėsime nuo to, kur baigėme [ankstesnėje pamokoje](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb), kur analizavome duomenų rinkinį, sukūrėme daug vizualizacijų ir filtravome duomenų rinkinį pagal dominančius stebėjimus. Būtinai peržiūrėkite!

Šiam moduliui atlikti reikės kelių paketų. Juos galite įdiegti naudodami: `install.packages(c('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork'))`

Arba žemiau pateiktas scenarijus patikrina, ar turite reikalingus paketus šiam moduliui atlikti, ir, jei trūksta, juos įdiegia.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))

pacman::p_load('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork')


Pradėkime!

## 1. Šokis su duomenimis: Susiaurinkime iki 3 populiariausių muzikos žanrų

Tai yra apžvalga to, ką darėme ankstesnėje pamokoje. Pasinerkime į duomenis!


In [ ]:
# Load the core tidyverse and make it available in your current R session
library(tidyverse)

# Import the data into a tibble
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/5-Clustering/data/nigerian-songs.csv", show_col_types = FALSE)

# Narrow down to top 3 popular genres
nigerian_songs <- df %>% 
  # Concentrate on top 3 genres
  filter(artist_top_genre %in% c("afro dancehall", "afropop","nigerian pop")) %>% 
  # Remove unclassified observations
  filter(popularity != 0)



# Visualize popular genres using bar plots
theme_set(theme_light())
nigerian_songs %>%
  count(artist_top_genre) %>%
  ggplot(mapping = aes(x = artist_top_genre, y = n,
                       fill = artist_top_genre)) +
  geom_col(alpha = 0.8) +
  paletteer::scale_fill_paletteer_d("ggsci::category10_d3") +
  ggtitle("Top genres") +
  theme(plot.title = element_text(hjust = 0.5))


🤩 Tai pavyko puikiai!

## 2. Daugiau duomenų tyrinėjimo.

Kaip švarūs yra šie duomenys? Patikrinkime išskirtis naudodami dėžutės diagramas. Susitelksime į skaitines stulpelius su mažiau išskirčių (nors galėtumėte pašalinti išskirtis). Dėžutės diagramos gali parodyti duomenų diapazoną ir padėti pasirinkti, kuriuos stulpelius naudoti. Atkreipkite dėmesį, kad dėžutės diagramos nerodo dispersijos, kuri yra svarbus gero klasterizuojamo duomenų elemento aspektas. Daugiau informacijos rasite [šiame aptarime](https://stats.stackexchange.com/questions/91536/deduce-variance-from-boxplot).

[Dėžutės diagramos](https://en.wikipedia.org/wiki/Box_plot) naudojamos grafiškai pavaizduoti `skaitinių` duomenų pasiskirstymą, todėl pradėkime nuo *visų skaitinių stulpelių pasirinkimo* kartu su populiariais muzikos žanrais.


In [ ]:
# Select top genre column and all other numeric columns
df_numeric <- nigerian_songs %>% 
  select(artist_top_genre, where(is.numeric)) 

# Display the data
df_numeric %>% 
  slice_head(n = 5)


Ar matote, kaip pasirinkimo pagalbinė funkcija `where` palengvina darbą 💁? Daugiau tokių funkcijų galite rasti [čia](https://tidyselect.r-lib.org/).

Kadangi ruošiamės kurti dėžutės diagramą kiekvienam skaitiniam požymiui ir norime išvengti ciklų naudojimo, pertvarkykime savo duomenis į *ilgesnį* formatą, kuris leis mums pasinaudoti `facets` - subsiužetais, kurie kiekvienas rodo vieną duomenų pogrupį.


In [ ]:
# Pivot data from wide to long
df_numeric_long <- df_numeric %>% 
  pivot_longer(!artist_top_genre, names_to = "feature_names", values_to = "values") 

# Print out data
df_numeric_long %>% 
  slice_head(n = 15)


Daug ilgesnis! Dabar metas šiek tiek `ggplots`! Taigi, kokį `geom` naudosime?


In [ ]:
# Make a box plot
df_numeric_long %>% 
  ggplot(mapping = aes(x = feature_names, y = values, fill = feature_names)) +
  geom_boxplot() +
  facet_wrap(~ feature_names, ncol = 4, scales = "free") +
  theme(legend.position = "none")


Lengvai-gg!

Dabar matome, kad šie duomenys yra šiek tiek triukšmingi: stebėdami kiekvieną stulpelį kaip dėžės diagramą, galite pastebėti išskirtis. Galėtumėte peržiūrėti duomenų rinkinį ir pašalinti šias išskirtis, tačiau tai padarytų duomenis gana minimaliais.

Kol kas pasirinkime, kuriuos stulpelius naudosime klasterizavimo užduočiai. Pasirinkime skaitinius stulpelius su panašiais diapazonais. Galėtume užkoduoti `artist_top_genre` kaip skaitinį, bet šiuo metu jį praleisime.


In [ ]:
# Select variables with similar ranges
df_numeric_select <- df_numeric %>% 
  select(popularity, danceability, acousticness, loudness, energy) 

# Normalize data
# df_numeric_select <- scale(df_numeric_select)


## 3. K-means klasterizacijos skaičiavimas R

K-means klasterizaciją R galima apskaičiuoti naudojant įmontuotą funkciją `kmeans`, žr. `help("kmeans()")`. Funkcija `kmeans()` kaip pagrindinį argumentą priima duomenų rėmelį su visomis skaitinėmis stulpeliais.

Pirmasis žingsnis naudojant k-means klasterizaciją yra nurodyti klasterių skaičių (k), kuris bus sukurtas galutiniame sprendime. Žinome, kad iš duomenų rinkinio išskyrėme 3 muzikos žanrus, todėl pabandykime 3:


In [ ]:
set.seed(2056)
# Kmeans clustering for 3 clusters
kclust <- kmeans(
  df_numeric_select,
  # Specify the number of clusters
  centers = 3,
  # How many random initial configurations
  nstart = 25
)

# Display clustering object
kclust


Kmeans objektas turi kelis informacijos elementus, kurie gerai paaiškinti `help("kmeans()")`. Šiuo metu susitelkime į kelis iš jų. Matome, kad duomenys buvo suskirstyti į 3 klasterius, kurių dydžiai yra 65, 110, 111. Rezultate taip pat pateikiami klasterių centrai (vidurkiai) 3 grupėms pagal 5 kintamuosius.

Klasterizavimo vektorius yra klasterio priskyrimas kiekvienam stebėjimui. Naudokime funkciją `augment`, kad pridėtume klasterio priskyrimą prie pradinio duomenų rinkinio.


In [ ]:
# Add predicted cluster assignment to data set
augment(kclust, df_numeric_select) %>% 
  relocate(.cluster) %>% 
  slice_head(n = 10)


Puiku, mes ką tik suskirstėme savo duomenų rinkinį į 3 grupes. Taigi, kaip gerai veikia mūsų klasterizacija 🤷? Pažvelkime į `Silhouette score`.

### **Silhouette score**

[Silhouette analizė](https://en.wikipedia.org/wiki/Silhouette_(clustering)) gali būti naudojama tirti atstumą tarp susidariusių klasterių. Šis rodiklis svyruoja nuo -1 iki 1, ir jei rezultatas yra arti 1, klasteris yra tankus ir gerai atskirtas nuo kitų klasterių. Vertė, artima 0, rodo persidengiančius klasterius, kurių pavyzdžiai yra labai arti kaimyninių klasterių sprendimo ribos. [šaltinis](https://dzone.com/articles/kmeans-silhouette-score-explained-with-python-exam).

Vidutinio silueto metodas apskaičiuoja vidutinį stebėjimų siluetą skirtingoms *k* reikšmėms. Aukštas vidutinio silueto rodiklis rodo gerą klasterizaciją.

`silhouette` funkcija iš klasterių paketo naudojama vidutiniam silueto pločiui apskaičiuoti.

> Siluetas gali būti apskaičiuotas naudojant bet kokį [atstumą](https://en.wikipedia.org/wiki/Distance "Distance"), pavyzdžiui, [Euklido atstumą](https://en.wikipedia.org/wiki/Euclidean_distance "Euclidean distance") arba [Manhatano atstumą](https://en.wikipedia.org/wiki/Manhattan_distance "Manhattan distance"), apie kuriuos kalbėjome [ankstesnėje pamokoje](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb).


In [ ]:
# Load cluster package
library(cluster)

# Compute average silhouette score
ss <- silhouette(kclust$cluster,
                 # Compute euclidean distance
                 dist = dist(df_numeric_select))
mean(ss[, 3])


Mūsų rezultatas yra **.549**, taigi ties viduriu. Tai rodo, kad mūsų duomenys nėra ypač tinkami tokio tipo klasterizavimui. Pažiūrėkime, ar galime vizualiai patvirtinti šią prielaidą. [factoextra paketas](https://rpkgs.datanovia.com/factoextra/index.html) suteikia funkcijas (`fviz_cluster()`) klasterizavimo vizualizavimui.


In [ ]:
library(factoextra)

# Visualize clustering results
fviz_cluster(kclust, df_numeric_select)


Klasterių persidengimas rodo, kad mūsų duomenys nėra ypač tinkami tokio tipo klasterizavimui, tačiau tęskime.

## 4. Optimalus klasterių skaičiaus nustatymas

Pagrindinis klausimas, kuris dažnai kyla naudojant K-Means klasterizavimą, yra toks – neturint žinomų klasių etikečių, kaip sužinoti, į kiek klasterių reikia suskirstyti duomenis?

Vienas iš būdų tai išsiaiškinti yra naudoti duomenų pavyzdį, kad `sukurtume seriją klasterizavimo modelių` su didėjančiu klasterių skaičiumi (pvz., nuo 1 iki 10) ir įvertintume klasterizavimo metrikas, tokias kaip **Silhouette rodiklis.**

Nustatykime optimalų klasterių skaičių, apskaičiuodami klasterizavimo algoritmą skirtingoms *k* reikšmėms ir įvertindami **klasterių vidinę kvadratų sumą** (WCSS). Bendroji vidinė klasterių kvadratų suma (WCSS) matuoja klasterizavimo kompaktiškumą, ir mes norime, kad ji būtų kuo mažesnė – mažesnės reikšmės reiškia, kad duomenų taškai yra arčiau vienas kito.

Pažvelkime, kaip skirtingos `k` reikšmės nuo 1 iki 10 veikia šį klasterizavimą.


In [ ]:
# Create a series of clustering models
kclusts <- tibble(k = 1:10) %>% 
  # Perform kmeans clustering for 1,2,3 ... ,10 clusters
  mutate(model = map(k, ~ kmeans(df_numeric_select, centers = .x, nstart = 25)),
  # Farm out clustering metrics eg WCSS
         glanced = map(model, ~ glance(.x))) %>% 
  unnest(cols = glanced)
  

# View clustering rsulsts
kclusts


Dabar, kai turime bendrą klasterio vidinę kvadratų sumą (tot.withinss) kiekvienam klasterizavimo algoritmui su centru *k*, naudojame [alkūnės metodą](https://en.wikipedia.org/wiki/Elbow_method_(clustering)), kad rastume optimalų klasterių skaičių. Šis metodas apima WCSS vaizdavimą kaip klasterių skaičiaus funkciją ir [kreivės alkūnės taško](https://en.wikipedia.org/wiki/Elbow_of_the_curve "Elbow of the curve") pasirinkimą kaip klasterių skaičių, kurį reikia naudoti.


In [ ]:
set.seed(2056)
# Use elbow method to determine optimum number of clusters
kclusts %>% 
  ggplot(mapping = aes(x = k, y = tot.withinss)) +
  geom_line(size = 1.2, alpha = 0.8, color = "#FF7F0EFF") +
  geom_point(size = 2, color = "#FF7F0EFF")


Grafikas rodo didelį WCSS sumažėjimą (didesnį *suglaudimą*), kai klasterių skaičius padidėja nuo vieno iki dviejų, ir dar pastebimą sumažėjimą nuo dviejų iki trijų klasterių. Po to sumažėjimas tampa mažiau ryškus, sukuriant `alkūnę` 💪 grafike ties maždaug trimis klasteriais. Tai yra geras ženklas, kad du ar trys klasteriai yra pakankamai gerai atskirti duomenų taškai.

Dabar galime pereiti prie klasterizavimo modelio, kur `k = 3`:

> `pull()`: naudojamas ištraukti vieną stulpelį
>
> `pluck()`: naudojamas indeksuoti duomenų struktūras, tokias kaip sąrašai


In [ ]:
# Extract k = 3 clustering
final_kmeans <- kclusts %>% 
  filter(k == 3) %>% 
  pull(model) %>% 
  pluck(1)


final_kmeans


Puiku! Pažiūrėkime į gautus klasterius. Norėtumėte šiek tiek interaktyvumo naudojant `plotly`?


In [ ]:
# Add predicted cluster assignment to data set
results <-  augment(final_kmeans, df_numeric_select) %>% 
  bind_cols(df_numeric %>% select(artist_top_genre)) 

# Plot cluster assignments
clust_plt <- results %>% 
  ggplot(mapping = aes(x = popularity, y = danceability, color = .cluster, shape = artist_top_genre)) +
  geom_point(size = 2, alpha = 0.8) +
  paletteer::scale_color_paletteer_d("ggthemes::Tableau_10")

ggplotly(clust_plt)


Galbūt būtume tikėjęsi, kad kiekvienas klasteris (vaizduojamas skirtingomis spalvomis) turės aiškiai atskirtus žanrus (vaizduojamus skirtingomis formomis).

Pažvelkime į modelio tikslumą.


In [ ]:
# Assign genres to predefined integers
label_count <- results %>% 
  group_by(artist_top_genre) %>% 
  mutate(id = cur_group_id()) %>% 
  ungroup() %>% 
  summarise(correct_labels = sum(.cluster == id))


# Print results  
cat("Result:", label_count$correct_labels, "out of", nrow(results), "samples were correctly labeled.")

cat("\nAccuracy score:", label_count$correct_labels/nrow(results))


Šio modelio tikslumas nėra blogas, tačiau nėra ir puikus. Gali būti, kad duomenys nėra tinkami K-Means klasterizacijai. Šie duomenys yra per daug nesubalansuoti, per mažai koreliuoti, o stulpelių reikšmės turi per didelę variaciją, kad būtų galima efektyviai sukurti klasterius. Iš tiesų, susiformavę klasteriai greičiausiai yra stipriai paveikti arba iškreipti trijų aukščiau apibrėžtų žanrų kategorijų.

Nepaisant to, tai buvo puikus mokymosi procesas!

Scikit-learn dokumentacijoje galite pamatyti, kad modelis, kurio klasteriai nėra gerai apibrėžti, turi „variacijos“ problemą:

<p >
   <img src="../../images/problems.png"
   width="500"/>
   <figcaption>Infografika iš Scikit-learn</figcaption>



## **Variacija**

Variacija apibrėžiama kaip „vidutinė kvadratinių skirtumų nuo vidurkio reikšmė“ [šaltinis](https://www.mathsisfun.com/data/standard-deviation.html). Šio klasterizacijos problemos kontekste tai reiškia, kad mūsų duomenų rinkinio skaičiai linkę per daug nukrypti nuo vidurkio.

✅ Tai puikus momentas pagalvoti apie visus būdus, kaip galėtumėte išspręsti šią problemą. Gal šiek tiek pakoreguoti duomenis? Naudoti kitus stulpelius? Pasirinkti kitą algoritmą? Užuomina: Pabandykite [normalizuoti savo duomenis](https://www.mygreatlearning.com/blog/learning-data-science-with-k-means-clustering/) ir išbandyti kitus stulpelius.

> Pabandykite šį '[variacijos skaičiuoklę](https://www.calculatorsoup.com/calculators/statistics/variance-calculator.php)', kad geriau suprastumėte šią sąvoką.

------------------------------------------------------------------------

## **🚀Iššūkis**

Praleiskite šiek tiek laiko su šiuo užrašų knygeliu, koreguodami parametrus. Ar galite pagerinti modelio tikslumą, dar labiau išvalydami duomenis (pavyzdžiui, pašalindami anomalijas)? Galite naudoti svorius, kad tam tikriems duomenų pavyzdžiams suteiktumėte daugiau reikšmės. Ką dar galite padaryti, kad sukurtumėte geresnius klasterius?

Užuomina: Pabandykite normalizuoti savo duomenis. Užrašų knygelėje yra komentaruose pateiktas kodas, kuris prideda standartinį mastelio keitimą, kad duomenų stulpeliai būtų panašesni pagal diapazoną. Pastebėsite, kad nors silueto balas sumažėja, „kinkas“ alkūnės grafike išsilygina. Taip yra todėl, kad palikus duomenis nenormalizuotus, mažesnės variacijos duomenys turi didesnę įtaką. Daugiau apie šią problemą skaitykite [čia](https://stats.stackexchange.com/questions/21222/are-mean-normalization-and-feature-scaling-needed-for-k-means-clustering/21226#21226).

## [**Po paskaitos testas**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/30/)

## **Apžvalga ir savarankiškas mokymasis**

-   Pažvelkite į K-Means simuliatorių [tokį kaip šis](https://user.ceng.metu.edu.tr/~akifakkus/courses/ceng574/k-means/). Galite naudoti šį įrankį, kad vizualizuotumėte pavyzdinius duomenų taškus ir nustatytumėte jų centroidus. Galite redaguoti duomenų atsitiktinumą, klasterių skaičių ir centroidų skaičių. Ar tai padeda jums geriau suprasti, kaip duomenys gali būti grupuojami?

-   Taip pat peržiūrėkite [šį K-Means vadovą](https://stanford.edu/~cpiech/cs221/handouts/kmeans.html) iš Stanfordo.

Norite išbandyti savo naujai įgytus klasterizacijos įgūdžius su duomenų rinkiniais, kurie gerai tinka K-Means klasterizacijai? Peržiūrėkite:

-   [Mokykite ir vertinkite klasterizacijos modelius](https://rpubs.com/eR_ic/clustering) naudodami Tidymodels ir kitus įrankius

-   [K-Means klasterizacijos analizė](https://uc-r.github.io/kmeans_clustering), UC Business Analytics R programavimo vadovas

- [K-Means klasterizacija su tvarkingų duomenų principais](https://www.tidymodels.org/learn/statistics/k-means/)

## **Užduotis**

[Bandykite skirtingus klasterizacijos metodus](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/2-K-Means/assignment.md)

## DĖKOJAME:

[Jen Looper](https://www.twitter.com/jenlooper) už originalios šio modulio Python versijos sukūrimą ♥️

[`Allison Horst`](https://twitter.com/allison_horst/) už nuostabių iliustracijų kūrimą, kurios padaro R labiau patrauklų ir įdomų. Daugiau iliustracijų rasite jos [galerijoje](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

Sėkmingo mokymosi,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambassador.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="500"/>
   <figcaption>Kūrinys @allison_horst</figcaption>



---

**Atsakomybės apribojimas**:  
Šis dokumentas buvo išverstas naudojant AI vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, prašome atkreipti dėmesį, kad automatiniai vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Kritinei informacijai rekomenduojama naudoti profesionalų žmogaus vertimą. Mes neprisiimame atsakomybės už nesusipratimus ar klaidingus interpretavimus, atsiradusius dėl šio vertimo naudojimo.
